In [1]:
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# create list of multiple dataframes containing participants' data
path =r'C:\Users\lttso\Desktop\test_data'
filenames = glob.glob(path + "/*.csv")
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))

In [3]:
#shows an example of pure df to work with
dfs[0]

,block_n,trial_n,training_trials,target1,target2,back_to_center,num_of_targets,condition,stim0,x_0,...,y_targ,targ_selected,valid_click,participant,age,gender,date,expName,psychopyVersion,Unnamed: 42
0,0,0,1,red_cc,green_cc,False,4,feat,green_cc,88.798470,...,-78.508362,True,True,Testsub1,23,f,2022_Aug_10_1244,foraging_circle,2021.2.3,NaN
1,0,0,1,red_cc,green_cc,False,3,feat,green_cc,46.377595,...,-151.933354,True,True,Testsub1,23,f,2022_Aug_10_1244,foraging_circle,2021.2.3,NaN
2,0,0,1,red_cc,green_cc,False,4,feat,yellow_cc,93.587744,...,-73.807224,True,True,Testsub1,23,f,2022_Aug_10_1244,foraging_circle,2021.2.3,NaN
3,0,0,1,red_cc,green_cc,False,3,feat,yellow_cc,137.414418,...,-80.483495,True,True,Testsub1,23,f,2022_Aug_10_1244,foraging_circle,2021.2.3,NaN
4,0,0,1,red_cc,green_cc,False,2,feat,green_cc,49.157680,...,-1.557801,True,True,Testsub1,23,f,2022_Aug_10_1244,foraging_circle,2021.2.3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2070,3,31,0,yellow_cc,blue_cc,False,4,feat,blue_cc,495.944003,...,-465.911057,True,True,Testsub1,23,f,2022_Aug_10_1244,foraging_circle,2021.2.3,NaN
2071,3,31,0,yellow_cc,blue_cc,True,3,feat,green_cc,87.676453,...,NaN,NaN,False,Testsub1,23,f,2022_Aug_10_1244,foraging_circle,2021.2.3,NaN
2072,3,31,0,yellow_cc,blue_cc,True,3,feat,green_cc,87.676453,...,-82.203575,True,True,Testsub1,23,f,2022_Aug_10_1244,foraging_circle,2021.2.3,NaN
2073,3,31,0,yellow_cc,blue_cc,False,5,feat,yellow_cc,137.178073,...,-73.044200,True,True,Testsub1,23,f,2022_Aug_10_1244,foraging_circle,2021.2.3,NaN


In [4]:
# Create a list to store cleaned dataframes
dfs_cleaned = []

# Get age and gender for further count
gender = []
age = []

#create dictionary with error rates
count = 1
error_rates_feat = dict()
error_rates_conj = dict()
error_rates = dict()
for df in dfs:
    # record age and gender for further count
    gen = df.gender.unique()
    a = df.age.unique()
    gender.append(gen[0])
    age.append(a[0])
    
    #clean df from unrelated data & columns
    df = df[df.training_trials != 1]
    df = df[df.valid_click != False]
    df = df.drop(['gender', 'age', 'date', 
                  'psychopyVersion', 'valid_click', 'expName'], axis=1) # add "Unnamed: 42" for lab data
    
    #create dfs for feature and conjunction
    df_feat = df[df['condition'] == 'feat']
    df_conj = df[df['condition'] == 'conj']
    
    
    error_selections = df.loc[(df['targ_selected'] == False), 'trial_n'].tolist()
    # calculate error rates for feture 
    actual_num_of_trials = 32 * 4
    error_select_feat = df_feat.loc[(df_feat['targ_selected'] == False), 'trial_n'].tolist()
    error_selections_feat = len(error_select_feat)
    rates_feat = 100 - (abs(error_selections_feat - actual_num_of_trials) / actual_num_of_trials * 100)
    #store error rates in dict('participant_n':error rate)
    error_rates_feat.update({str(count): rates_feat})
    
    # calculate error rates for conj 
    error_select_conj = df_conj.loc[(df_conj['targ_selected'] == False), 'trial_n'].tolist()
    error_selections_conj = len(error_select_conj)
    rates_conj = 100 - (abs(error_selections_conj - actual_num_of_trials) / actual_num_of_trials * 100)
    #store error rates in dict('participant_n':error rate)
    error_rates_conj.update({str(count): rates_conj})
    
    # calculate error rates for both conditions
    trials = 32 * 2 * 4
    error_selections_n = len(error_selections)
    rates = 100 - (abs(error_selections_n - trials) / trials * 100)
    error_rates.update({str(count): rates})
    
    count += 1
    
#     #remove error trials
#     df = df[~df['trial_n'].isin(error_selections)]
    
    #create lists of dataframes
    dfs_cleaned.append(df)

In [5]:
error_rates_feat # in future make a simple t-test to compare feat/conj error rates

{'1': 10.9375, '2': 6.25, '3': 7.03125, '4': 0.78125, '5': 9.375}

In [6]:
error_rates_conj

{'1': 17.96875, '2': 5.46875, '3': 1.5625, '4': 0.78125, '5': 20.3125}

In [7]:
error_rates

{'1': 14.453125, '2': 5.859375, '3': 4.296875, '4': 0.78125, '5': 14.84375}

In [8]:
dfs_cleaned[0]

,block_n,trial_n,training_trials,target1,target2,back_to_center,num_of_targets,condition,stim0,x_0,...,y_resp,click_time,direction,switch,selected_stim,x_targ,y_targ,targ_selected,participant,Unnamed: 42
167,0,0,0,red_cc,green_sq,False,4,conj,green_sq,89.624846,...,-74.0,0.717232,4.0,1.0,red_cc,-44.689770,-78.337172,True,Testsub1,NaN
168,0,0,0,red_cc,green_sq,False,3,conj,red_cc,49.319420,...,-78.0,0.566876,0.0,0.0,red_cc,49.319420,-72.951860,True,Testsub1,NaN
169,0,0,0,red_cc,green_sq,False,4,conj,green_sq,138.500886,...,-74.0,0.550609,3.0,0.0,red_cc,-49.681748,-82.363355,True,Testsub1,NaN
170,0,0,0,red_cc,green_sq,False,4,conj,red_cc,41.084824,...,-90.0,0.517183,0.0,0.0,red_cc,41.084824,-81.977850,True,Testsub1,NaN
171,0,0,0,red_cc,green_sq,False,4,conj,green_sq,136.839798,...,-71.0,0.617372,NaN,NaN,red_sq,-43.487535,-75.539938,False,Testsub1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2069,3,31,0,yellow_cc,blue_cc,False,3,feat,red_cc,454.814598,...,-392.0,0.766920,5.0,0.0,yellow_cc,400.915028,-389.020554,True,Testsub1,NaN
2070,3,31,0,yellow_cc,blue_cc,False,4,feat,blue_cc,495.944003,...,-457.0,0.984269,5.0,0.0,yellow_cc,451.162368,-465.911057,True,Testsub1,NaN
2072,3,31,0,yellow_cc,blue_cc,True,3,feat,green_cc,87.676453,...,-92.0,0.950739,5.0,0.0,yellow_cc,44.788289,-82.203575,True,Testsub1,NaN
2073,3,31,0,yellow_cc,blue_cc,False,5,feat,yellow_cc,137.178073,...,-83.0,0.734102,0.0,0.0,yellow_cc,137.178073,-73.044200,True,Testsub1,NaN


## Merged dataset

In [9]:
#merge dataframes in one big df
df = pd.concat(dfs_cleaned)

In [10]:
df = df.reset_index(drop=True)
df = df.drop('Unnamed: 42', axis=1) # just a strange column appered from one data

In [11]:
df

,block_n,trial_n,training_trials,target1,target2,back_to_center,num_of_targets,condition,stim0,x_0,...,x_resp,y_resp,click_time,direction,switch,selected_stim,x_targ,y_targ,targ_selected,participant
0,0,0,0,red_cc,green_sq,False,4,conj,green_sq,89.624846,...,-39.0,-74.0,0.717232,4.0,1.0,red_cc,-44.689770,-78.337172,True,Testsub1
1,0,0,0,red_cc,green_sq,False,3,conj,red_cc,49.319420,...,59.0,-78.0,0.566876,0.0,0.0,red_cc,49.319420,-72.951860,True,Testsub1
2,0,0,0,red_cc,green_sq,False,4,conj,green_sq,138.500886,...,-70.0,-74.0,0.550609,3.0,0.0,red_cc,-49.681748,-82.363355,True,Testsub1
3,0,0,0,red_cc,green_sq,False,4,conj,red_cc,41.084824,...,16.0,-90.0,0.517183,0.0,0.0,red_cc,41.084824,-81.977850,True,Testsub1
4,0,0,0,red_cc,green_sq,False,4,conj,green_sq,136.839798,...,-54.0,-71.0,0.617372,NaN,NaN,red_sq,-43.487535,-75.539938,False,Testsub1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8821,3,31,0,red_cc,green_sq,False,3,conj,red_cc,-3.853679,...,3.0,-10.0,0.632550,0.0,1.0,red_cc,-3.853679,1.558571,True,test_sub_5
8822,3,31,0,red_cc,green_sq,False,3,conj,red_sq,94.783756,...,-86.0,9.0,0.516050,3.0,1.0,green_sq,-92.279415,2.970947,True,test_sub_5
8823,3,31,0,red_cc,green_sq,False,5,conj,green_sq,-0.623598,...,-111.0,90.0,0.566131,2.0,1.0,red_cc,-130.146907,76.501629,True,test_sub_5
8824,3,31,0,red_cc,green_sq,False,5,conj,red_cc,-45.345008,...,-80.0,141.0,0.549312,1.0,1.0,green_sq,-88.642081,152.472224,True,test_sub_5


In [12]:
# function to calculate the number of targets per selection
def targets_per_selection(row):
    num_of_targets = 0
    for i in range(0, 6):
        if row['target1'] == row['stim' + str(i)] or row['target2'] == row['stim' + str(i)]:
            num_of_targets += 1
    return num_of_targets

In [13]:
# New column with the number of targets in selection patch  
df['num_of_targets'] = df.apply(lambda row: targets_per_selection(row), axis=1)
# New column with the ratio of targets to distractors in selection patch
df['ratio'] = df.apply(lambda row: row.num_of_targets / 6, axis = 1)

In [14]:
# add direction value for errors
df_false = df.loc[(df['targ_selected'] == False)]
for n in df_false.index:
    for i in range(0, 5):
        if df_false['x_targ'][n] == df_false['x_'+ str(i)][n] and df_false['y_targ'][n] == df_false['y_'+ str(i)][n]:
            df['direction'][n] = i

C:\Users\lttso\AppData\Local\Temp\ipykernel_4900\260873606.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['direction'][n] = i


In [15]:
#clean data from error selections
df_no_errors = df[df.targ_selected != False]

In [16]:
# make a csv file of data with/without errors
df.to_csv (r'C:\Users\lttso\Desktop\with_errors_foraging_ratio.csv', index = False,  mode='a', header=True)
df_no_errors.to_csv (r'C:\Users\lttso\Desktop\no_errors_foraging_ratio.csv', index = False,  mode='a', header=True)